# Feature Extraction: AirUI

## Goal

In this notebook we aim to segment and form mel-frequency spectrograms for each of them. The output images will be of variable length to account for different lengths of each gesture, and the CNN model will crop/pad them as necessary.

## Tasks/Implementation

- [x] Create file arp system to gain access to all necessary audio files.
- [x] Create a function that takes in a file path and returns the segmented numpy arrays.
- [x] Create a function that takes in the list of numpy arrays and produces a list of mel-frequency spectrograms.
- [x] Create export function that takes in class name and list of spectrograms (as numpy arrays) and outputs it into the appropriate directory.
- [x] Create master process function to automate the entire process.
- [x] Run the process, send results to team via Drive. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# IMPORT BOX #
import os
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas
from scipy.io import wavfile
import scipy.io
import random

import librosa
import librosa.display
import IPython.display as ipd

In [ ]:
"""Constants"""

base_path = 'drive/MyDrive/\'Third Year\'/ECE324/\'Mickey Mouse AI Club\'/\'03 Data Collection\'/Cropped_Stems'
base_path2 = 'drive/MyDrive/Third Year/ECE324/Mickey Mouse AI Club/03 Data Collection/Cropped_Stems'

folders = ['Circle_Scratches',
 'Fingernail_Taps',
 'Fingertip_Taps',
 'Silences',
 'Vertical_Scratches',
 'W_Scratches']

Subjects = ['Aman Bhargava(1).wav',
 'Andre Fu.wav',
 'Brian C.wav',
 'Bryen Hume.wav',
 'David Carnaffan.wav',
 'Divy Raval.wav',
 'Elysia Harrison.wav',
 'Gerry Chen.wav',
 'Hairan Zheng.wav',
 'Jayden Lefebvre.wav',
 'Eli Scott.wav',
 'Kelvin Cui.wav',
 'Michal Fishkin.wav',
 'pathwaystopeaceyoga@hotmail.com.wav',
 'Shayla Martin-Fenn.wav',
 'Siddharth Vijay.wav',
 'Kathryn Carnaffan.wav',
 'Rachel Green.wav',
 'Spencer Teetaert.wav',
 'Adam Carnaffan.wav',
 'Aidan Bowers.wav',
 'Alice Zhou.wav',
 'Alice Zhou(1).wav',
 'Aman Bhargava.wav']

seg_lens = [1, 0.75, 0.75, 0.75, 0.75, 1.5] # length in seconds of
                                            # each sample type.

FS_global = 44100 # Global value for sampling rate (set in DAW)

num_mels_global = 100

In [ ]:
def get_file_list(base_path, folder_name):
    """
    Gets a list of file names (strings) in the basepath/folder_name/ 
    directory.
    """
    path = base_path + '/' + folder_name
    stream = os.popen('ls {}'.format(path))
    output = stream.read()
    file_names = output.split('\n')[:-1]
    return file_names

In [ ]:
def segment_audio(FS, waves, in_len, out_len=0.75):
    """ 
    Returns a list of 1D numpy arrays of `out_len` snippets of samples from an input audio file.
    
    Keyword arguments:
    
    FS --         Sampling rate.
    waves --      Input audio waveforms. 
    in_len --     Length (in SECONDS) of each gesture in the recording (Sample BPM/60).
    
    Optional arguments: 
    
    out_len --    Desired length (in SECONDS) of the gestures in the output (will be symmetrically trimmed).
                  Must be less than or equal to `in_len`.
                  Defaults to 0.75 seconds.
    """
    num_samples = int(len(waves)/(FS*in_len)) # Assuming one per second.
    # TODO: Add contingency for 1 vs. 2-channel input waveforms. 
    wave = (waves[:,0] + waves[:,1])*0.5 # 
    sample_list = []
    
    pad = (in_len - out_len)/2
    
    for i in range(num_samples):
        wave_start = int(i*FS*in_len+pad*FS)
        wave_end = int((i+1)*FS*in_len-pad*FS)
        sample_list.append(wave[wave_start:wave_end])

    return sample_list

In [ ]:
def subjects_include_file_name (file_name, subjects):
  for subject in subjects: 
    if subject in file_name:
      return True
  return False

In [ ]:
def get_full_segment_list(base_path, folder_name, seg_len, test_subjects):
    """
    Returns a list of numpy audio segments of length `seg_len`
    (in seconds) extracted from ALL audio files inside of 
    /base_path/folder_name/
    """
    file_names = get_file_list(base_path, folder_name)
    
    ret_train_list = []
    ret_test_list = []
    
    for name in file_names:
        file_path = base_path2 + '/' + folder_name+'/' +name
        FS, audio = wavfile.read(file_path)
        seg_list = segment_audio(FS, audio, seg_len, out_len=seg_len)
        if subjects_include_file_name(name, test_subjects):
          ret_test_list += seg_list
        else: 
          ret_train_list += seg_list
    return ret_train_list, ret_test_list
  

In [ ]:
def seg_list_to_spec_list(seg_list, num_mels, FS):
    spec_list = []
    for seg in seg_list:
        filter_banks = librosa.filters.mel(n_fft=2048, sr=FS, n_mels=num_mels)
        mel_spectrogram = librosa.feature.melspectrogram(seg, sr=FS, n_fft=2048, hop_length=512, n_mels=num_mels)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
        spec_list.append(log_mel_spectrogram)
        
    return spec_list

In [ ]:
def dump_spec_list_to_folder(spec_list, name, path='../Full_Dataset/Spectrograms/'):
    pth = path+name
    
    for i, spec in enumerate(spec_list):
        plt.imsave('{}/{}.png'.format(pth,i), spec, cmap='gray')
    
    print('Done saving all images in {}'.format(name))

In [143]:
def main():
    test_subjects = random.sample(Subjects, 6)     #6 people to the test set, 18 people to the training set
    print('The selected participants in the test set are:') 
    print (test_subjects)   
    for i in range(len(folders)):
        print('Starting {}...'.format(folders[i]))
        seg_list_train, seg_list_test = get_full_segment_list(base_path, folders[i], seg_lens[i], test_subjects)
        print('Finished Segmenting {}...'.format(folders[i]))
        spec_list_train = seg_list_to_spec_list(seg_list_train, num_mels_global, FS_global)
        spec_list_test = seg_list_to_spec_list(seg_list_test, num_mels_global, FS_global)
        print('Finished Computing Mel-Spectrograms for {}'.format(folders[i]))
        
        dump_spec_list_to_folder(spec_list_train, folders[i], path='drive/MyDrive/Third Year/ECE324/Mickey Mouse AI Club/04 Datasets/dataset_v02/train/')
        dump_spec_list_to_folder(spec_list_test, folders[i], path='drive/MyDrive/Third Year/ECE324/Mickey Mouse AI Club/04 Datasets/dataset_v02/test/')
        print("")

In [146]:
main()

The selected participants in the test set are:
['Alice Zhou.wav', 'Aidan Bowers.wav', 'Adam Carnaffan.wav', 'Alice Zhou(1).wav', 'David Carnaffan.wav', 'Andre Fu.wav']
Starting Circle_Scratches...
Finished Segmenting Circle_Scratches...
Finished Computing Mel-Spectrograms for Circle_Scratches
Done saving all images in Circle_Scratches
Done saving all images in Circle_Scratches

Starting Fingernail_Taps...
Finished Segmenting Fingernail_Taps...
Finished Computing Mel-Spectrograms for Fingernail_Taps
Done saving all images in Fingernail_Taps
Done saving all images in Fingernail_Taps

Starting Fingertip_Taps...
Finished Segmenting Fingertip_Taps...
Finished Computing Mel-Spectrograms for Fingertip_Taps
Done saving all images in Fingertip_Taps
Done saving all images in Fingertip_Taps

Starting Silences...
Finished Segmenting Silences...
Finished Computing Mel-Spectrograms for Silences
Done saving all images in Silences
Done saving all images in Silences

Starting Vertical_Scratches...
Fini